In [61]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains import LLMChain
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel
import time

In [62]:
with open('system_prompt.txt', 'r') as file:
    system_prompt = file.read()
system_prompt = system_prompt.replace("{", "{{").replace("}", "}}")

In [63]:
class StepResponse(BaseModel):
    title: str = Field(description="title of the step")
    content: str = Field(description="content of the step")
    confidence: str = Field(description="model confidence level for current step")
    next_action: str = Field(description="next action performed by the model")

In [64]:
parser = JsonOutputParser(pydantic_object=StepResponse)
# Define the prompt template
template = f"""{system_prompt}
User Input: {{user_message}}
"""
# Create the prompt
prompt = ChatPromptTemplate.from_template(template)
print(prompt.input_variables)
model = OllamaLLM(model="llama3.1")
# Build the chain
chain = prompt | model | parser
user_message = "How many r are in strawberry"

['user_message']


In [65]:
def generate_chain_of_thought(user_message):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": "Thank you! I will now think step by step following my instructions, starting at the beginning after decomposing the problem."}
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        response = chain.invoke({"user_message": user_message})
        step_data = response
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        print(step_data)

        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))

        if step_data['next_action'] == 'final_answer':
            break

        # Add the step to the messages to continue reasoning
        user_message = step_data['content']
        step_count += 1

    # Generate the final answer
    steps.append(("Final Answer", step_data['content'], thinking_time))

    return steps, total_thinking_time

In [66]:
user_message = "How many r are in strawberry?"
steps, total_time = generate_chain_of_thought(user_message)

{'title': 'Initial Problem Analysis', 'content': "The problem asks for the number of 'r's in the word 'strawberry'. This is a straightforward linguistic query. I will consider multiple approaches to arrive at an answer.", 'confidence': 95, 'next_action': 'continue'}
{'title': 'Identify Linguistic Components', 'content': "The word 'strawberry' consists of 10 letters: s-t-r-a-w-b-e-r-r-y. To count the number of 'r's, I'll break down the word into its constituent parts and examine each letter individually.", 'confidence': 95, 'next_action': 'continue'}
{'title': 'Break Down Word Components', 'content': "The given word is 'strawberry', which can be separated into individual letters: s-t-r-a-w-b-e-r-r-y. To identify the number of 'r's, I will examine each letter position.", 'confidence': 95, 'next_action': 'continue'}
{'title': "Counting R's in the Word 'strawberry'", 'content': "The word 'strawberry' has 9 letters: s-t-r-a-w-b-e-r-r-y. To count the number of 'r's, I'll examine each positio

KeyboardInterrupt: 

In [ ]:
# Print the steps
for step in steps:
    print(f"{step[0]}\n{step[1]}\nTime Taken: {step[2]:.2f} seconds\n")

print(f"Total Thinking Time: {total_time:.2f} seconds")